In [1]:
%pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 47.5 MB/s eta 0:00:0000:010:01
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 39.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install pyarrow fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 44.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.9/579.9 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 29.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [37]:
import pandas as pd
import pyspark

In [38]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [39]:
spark=SparkSession.builder.config("spark.driver.memory","12g").config("spark.memory.offHeap.enabled","true") .config("spark.memory.offHeap.size","10g").appName('task3_ml').getOrCreate()

In [40]:
spark

In [41]:
file_path = './ml_input_data.parquet/*.parquet'
pyspark_df = spark.read.parquet(file_path, header=True, inferSchema=True)
pyspark_df.show(3)

+------+----------+--------------+---------------------+
|Symbol|Volume_New|vol_moving_avg|adj_close_rolling_med|
+------+----------+--------------+---------------------+
|    AB|  471200.0|          null|                  NaN|
|    AB|  430000.0|      471200.0|           0.03649914|
|    AB|  430000.0|      450600.0|           0.03694973|
+------+----------+--------------+---------------------+
only showing top 3 rows



In [42]:
pandas_df = pyspark_df.toPandas()
pandas_df.head(3)

,Symbol,Volume_New,vol_moving_avg,adj_close_rolling_med
0,AB,471200.0,NaN,NaN
1,AB,430000.0,471200.0,0.036499
2,AB,430000.0,450600.0,0.036950


In [43]:
#Missing Values
pandas_df.dropna(axis = 0, inplace = True)
pandas_df.head(3)

,Symbol,Volume_New,vol_moving_avg,adj_close_rolling_med
1,AB,430000.0,471200.000000,0.036499
2,AB,430000.0,450600.000000,0.036950
3,AB,430000.0,443733.333333,0.037400


In [44]:
pandas_df.reset_index(drop = True, inplace= True)
pandas_df.head(3)

,Symbol,Volume_New,vol_moving_avg,adj_close_rolling_med
0,AB,430000.0,471200.000000,0.036499
1,AB,430000.0,450600.000000,0.036950
2,AB,430000.0,443733.333333,0.037400


In [45]:
x_df = pandas_df[['Symbol', 'vol_moving_avg', 'adj_close_rolling_med']]
y_df = pandas_df['Volume_New']

In [46]:
x_df.head(3)

,Symbol,vol_moving_avg,adj_close_rolling_med
0,AB,471200.000000,0.036499
1,AB,450600.000000,0.036950
2,AB,443733.333333,0.037400


In [10]:
%pip install scikit-learn 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 44.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.7/28.7 MB 48.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 49.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [47]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train, y_test = train_test_split(x_df,y_df, test_size = 0.2, stratify=x_df[["Symbol"]])

In [48]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers= [ ("encoder", OneHotEncoder(),[0])], remainder="passthrough")


In [49]:
symbol_encoder =  ct.fit(x_df) 

In [17]:
import pickle
filename = './sklearn_models/symbol_encoder.sav'
pickle.dump(symbol_encoder, open(filename, 'wb'))

In [50]:
import numpy as np
x_train =  symbol_encoder.transform(x_train) 
x_test =  symbol_encoder.transform(x_test) 

In [ ]:
"""
Linear regression modelling
"""

In [51]:
from sklearn.linear_model import LinearRegression 

In [52]:
LR = LinearRegression()
LR.fit(x_train  ,y_train)

LinearRegression()

In [21]:
filename = './sklearn_models/linear_regression.sav'
pickle.dump(LR, open(filename, 'wb'))

In [22]:
LR.coef_

array([ 4.51048426e-36,  7.54894711e-36, -1.42482674e-37, ...,
       -1.45882644e-36,  1.21037327e-24,  5.66534074e-16])

In [23]:
y_pred = LR.predict(x_test)

In [36]:
array_1 = np.array([['AB',1000,100]])
df = pd.DataFrame(array_1, columns=['Symbol', 'vol_moving_avg', 'adj_close_rolling_med'])

test_data = symbol_encoder.transform(df) 
y_pred = LR.predict(test_data)
y_pred[0] 

1102952.807354919

In [20]:
x_train 

<32940084x8045 sparse matrix of type '<class 'numpy.float64'>'
	with 98309945 stored elements in Compressed Sparse Row format>

In [25]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math

In [26]:
MAE = mean_absolute_error(y_test,y_pred) #4.21 

MSE = mean_squared_error(y_test,y_pred) #32.42 

RMSE = math.sqrt(MSE) #5.62

Ytest_Mean = y_test.mean()  #22.85   

print('Mean Absolute Error:', MAE)
print('Mean Squared Error:', MSE)
print('Root Mean Squared Error:', RMSE)
print('Mean:', Ytest_Mean)

def mean_absolute_percentage_error (y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean( np.abs((y_true - y_pred)/y_true)) * 100


MAPE = mean_absolute_percentage_error(y_test,y_pred) #48%

print('Mean Absolute Percentage Error:', MAPE)

print('Scatter Index (RMSE/Mean):', (RMSE/Ytest_Mean) * 100)

Mean Absolute Error: 1610514.567522084
Mean Squared Error: 193213797344765.38
Root Mean Squared Error: 13900136.594464293
Mean: 1101506.878487999
Mean Absolute Percentage Error: inf
Scatter Index (RMSE/Mean): 1261.9200901899528


/var/folders/pv/lpp89_3d53328ck1565vv7p40000gn/T/ipykernel_44509/2240912289.py:16: RuntimeWarning: divide by zero encountered in divide
  return np.mean( np.abs((y_true - y_pred)/y_true)) * 100


In [ ]:
"""
Random Forest Regression
"""

In [43]:
from sklearn.ensemble import RandomForestRegressor

In [46]:
RFR = RandomForestRegressor(n_estimators=200,max_depth = 20, random_state=33)

In [ ]:
RFR.fit(x_train,y_train)

In [ ]:
y_pred = RFR.predict(x_test)

In [ ]:
"""
Support Vector Regression
"""

In [23]:
from sklearn.svm import SVR

In [24]:
Regressor_SVR = SVR(kernel="rbf")

In [ ]:
Regressor_SVR.fit(x_train,y_train)